# 작사가 인공지능 만들기

## 1. 데이터 읽어오기

In [1]:
import glob
import os
import re
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


## 2. 데이터 정제

In [3]:
#어떠한 데이터가 있는지 확인하기 위헤 
print("Examples:\n", raw_corpus[:50])

Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face [Verse 1]', "Somethin' ain't right when we talkin'", "Somethin' ain't right when we talkin'", "Look like you hidin' your problems", 'Really you never was solid', 'No, you can\'t "son" me', "You won't never get to run me", 'Just when shit look out of reach', 'I reach back like one, three', 'Like one, three, yeah [Pre-Hook]', "That's when they smile in my face", 'Whole time they wanna take my place', 'Whole time they wanna take my place', 'Whole time they wanna take my place', 'Yeah, I know they wanna take my place', 'I can tell that love is fake', "I don't trust a word you say", 'How you wanna clique up after your mistakes?', "Look you in th

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

In [5]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜀.
    if len(sentence) == 0:
        continue
    if len(sentence.split(' ')) > 12: # <start>, <end>를 붙이기 전의 문장이 12개 이하로 split 된다면 token도 14개 이하가 될 것.
        continue
    if sentence[-1] == ':':
        continue
    if sentence[0] == '[' and sentence[-1] == ']': # [hook]과 같은 형태 정제해줌
        continue
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

In [6]:
print(len(corpus))

157652


### 2-2. tokenize화 하기

In [7]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus) # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 14)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [8]:
tensor, tokenizer = tokenize(corpus)

[[   5   91  103 ...   10   12    3]
 [   2   42  133 ...    0    0    0]
 [   2    5   40 ...    0    0    0]
 ...
 [   2  203    3 ...    0    0    0]
 [   2  437    9 ...    0    0    0]
 [   2    9 1613 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa6ff980cd0>


# 3. 평가 데이터셋 분리

In [9]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

In [10]:
#train_test_split사용해 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42)

In [11]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Test:", enc_val.shape)
print("Target Test:", dec_val.shape)


Source Train: (126121, 13)
Target Train: (126121, 13)
Source Test: (31531, 13)
Target Test: (31531, 13)


In [12]:
#하이퍼 파라미터 준비하기
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋 만들기
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

# 4. 인공지능 만들기

In [31]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        self.batchnorm_1 = tf.keras.layers.BatchNormalization()
        self.batchnorm_2 = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(0.1)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.batchnorm_1(out)
        out = self.rnn_2(out)
        out = self.batchnorm_2(out)
        out = self.dropout(out)
        out = self.linear(out)
        
        return out

In [32]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [33]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
492/492 [==============================] - 188s 370ms/step - loss: 3.7574
Epoch 2/10
492/492 [==============================] - 184s 372ms/step - loss: 2.7304
Epoch 3/10
492/492 [==============================] - 184s 373ms/step - loss: 2.2865
Epoch 4/10
492/492 [==============================] - 183s 372ms/step - loss: 1.9801
Epoch 5/10
492/492 [==============================] - 183s 372ms/step - loss: 1.7531
Epoch 6/10
492/492 [==============================] - 183s 371ms/step - loss: 1.5808
Epoch 7/10
492/492 [==============================] - 183s 372ms/step - loss: 1.4520
Epoch 8/10
492/492 [==============================] - 183s 371ms/step - loss: 1.3540
Epoch 9/10
492/492 [==============================] - 184s 373ms/step - loss: 1.2754
Epoch 10/10
492/492 [==============================] - 184s 373ms/step - loss: 1.2151


## 4-2. 모델 평가하기

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        #입력받은 문장의 텐서 입력
        predict = model(test_tensor)
        # 예측된 값 중 가장 높은 확률의 word index 뽑음. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 위의 word index 문장에 붙힘. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        #<end>나 maxlen에 도달하면 끝. 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a monster <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> love")

'<start> love is a beautiful thing <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> die")

'<start> die while america drink your blood <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> what")

'<start> what you want nixga what you what you want nixga <end> '

In [26]:
generate_text(model, tokenizer, init_sentence="<start> he knows")

'<start> he knows hes so fucking gifted <end> '

## 5. 회고록

1. loss값을 줄이는 것이 가장 관건인 과제였다.
    - batch normalization
    
    - dropout
    
    위의 두 가지 방법을 사용해 loss 값을 2.2 아래로 줄일 수 있었다. 

2. dropout에 관한 회고

    - 사실 dropout을 사용하기는 했지만, 정확히 어느 수준, 어느 곳에 사용해야 하는지에 대해 잘 알지 못했다. https://stats.stackexchange.com/questions/240305/where-should-i-place-dropout-layers-in-a-neural-network
    위 글을 참고해 0.1수준으로 맞춰주었다. output 이전에 위치해주는 것이 좋다고 한다. 
    
3. Batch Normalization에 관한 회고
    
    - input normalization의 방법 중에 하나이다. scaling 효과를 얻기 위해 써주었고, after fully connected or
    convolutional layers / before nonlinearity의 위치에 넣어주었다. 
    
4. 자연어에 대한 프로젝트는 처음이라 굉장히 어려웠다. 특히 tokennize의 개념이 맨 처음에 굉장히 어려웠다. 하지만, 천천히 따라가다 보니 어느정도
    따라갈 수 있었다. 하지만, 모델의 성능을 높이는 부문이 굉장히 어려웠다. 
    딥러닝의 개념을 배우고 있긴 하지만, 그것을 실질적으로 써보는 경험이 부족해 이론과 구현의 괴리가 심하다. 
    또한, 이렇게 큰 데이터 셋은 한번 돌리면 시간이 굉장히 많이 걸리는데, dropout의 비율이나 위치등의 hyperparameter handling 시에 
    모든 스텝을 다 일일히 트레이닝 시켜보고 기다려야 하는 것인지, 아니면 내가 모르는 방법이 있는 것인지 궁금하다. 